In [133]:
import scipy as sc
from scipy import stats
import numpy as np
import csv
import time
import networkx as nx 
import pandas as pd
from collections import defaultdict
c2a = np.loadtxt('c2a_small_sub.txt')
a2q = np.loadtxt('a2q_small_sub.txt')
c2q = np.loadtxt('c2q_small_sub.txt')


In [130]:
#----------------------------------------------------------#
#                     Function Part                        #
#----------------------------------------------------------#
def create_graph(data):
    G = nx.Graph()
    edges = data[:,0:2].copy()
    G.add_edges_from(edges)
    return G
def construct_graph(G_c2a,G_a2q,G_c2q):
    node_c2a = G_c2a.nodes
    node_a2q = G_a2q.nodes
    node_c2q = G_c2q.nodes

    node = list(node_c2a&node_a2q&node_c2q)

    g_c2a = G_c2a.subgraph(node)
    g_a2q = G_a2q.subgraph(node)
    g_c2q = G_c2q.subgraph(node)
    return g_c2a,g_a2q,g_c2q

def rec_dd():
    return defaultdict(rec_dd)

In [118]:
#----------------------------------------------------------#
#      Construct 1500-nodes Completed Network              #
#----------------------------------------------------------#
Total_1500_nodes = np.concatenate((c2a, a2q), axis=0)
Total_1500_nodes = np.concatenate((Total_1500_nodes,c2q),axis=0)
print(c2a.shape,c2q.shape, c2q.shape,Total_1500_nodes.shape, )
total_network = create_graph(Total_1500_nodes)
c2a_graph = create_graph(c2a)
a2q_graph = create_graph(a2q)
c2q_graph = create_graph(c2q)
# to get common nodes for each layer
g_c2a, g_a2q, g_c2q = construct_graph(c2a_graph,a2q_graph,c2q_graph)

print('Combined Graph:''number of edges:',total_network.number_of_edges(),'/number of nodes:',total_network.number_of_nodes(),'/is connected or not?:',nx.is_connected(total_network))
print('Graph:c2a','number of edges:',g_c2a.number_of_edges(),'/number of nodes:',g_c2a.number_of_nodes(),'/is connected or not?:',nx.is_connected(g_c2a))
print('Graph:a2q','number of edges:',g_a2q.number_of_edges(),'/number of nodes:',g_a2q.number_of_nodes(),'/is connected or not?:',nx.is_connected(g_a2q))
print('Graph:c2q','number of edges:',g_c2q.number_of_edges(),'/number of nodes:',g_c2q.number_of_nodes(),'/is connected or not?:',nx.is_connected(g_c2q))

connected_graph = max(nx.connected_component_subgraphs(total_network), key=len)
print('Connected Graph:''number of edges:',connected_graph.number_of_edges(),'/number of nodes:',connected_graph.number_of_nodes(),'/is connected or not?:',nx.is_connected(connected_graph))



(18965, 3) (4326, 3) (4326, 3) (27683, 3)
Combined Graph:number of edges: 5208 /number of nodes: 758 /is connected or not?: True
Graph:c2a number of edges: 2557 /number of nodes: 657 /is connected or not?: False
Graph:a2q number of edges: 2716 /number of nodes: 657 /is connected or not?: False
Graph:c2q number of edges: 1355 /number of nodes: 657 /is connected or not?: False
Connected Graph:number of edges: 5208 /number of nodes: 758 /is connected or not?: True


In [196]:
#----------------------------------------------------------#
#                Choose One Centrality                     #
#----------------------------------------------------------#
betweenness = nx.betweenness_centrality(connected_graph)
closeness = nx.closeness_centrality(connected_graph)
#----------------------------------------------------------#
#      Rank Centrality to Seperate 3 Groups                #
#----------------------------------------------------------#
sorted_betweeness = sorted(betweenness.items(), key=operator.itemgetter(1))
sorted_closeness = sorted(closeness.items(), key=operator.itemgetter(1))


In [218]:
t1 = np.percentile(list(betweenness.values()),33)
t2 = np.percentile(list(betweenness.values()),67)
print(t1, t2)
range(0,1.0/3*len(sorted_closeness))
# print(sorted_closeness)
# active users:23283,33213,54964,6309,20860
# normal users:19884,18091,65584,59546,105148
# inactive users:69401,104707,162154,152286,113682

0.00013014690515011543 0.0010812316392822174


list

In [189]:
#----------------------------------------------------------#
#                   Processing Timestamp                   #
#----------------------------------------------------------#
sorted_timestamp = sorted(Total_1500_nodes, key=lambda entry: entry[2])
timestamp_sorted_data = pd.DataFrame(sorted_timestamp)
# df[list("ABCD")] = df[list("ABCD")].astype(int)
lower_quantile, lower_medium_quantile,upper_medium_quantile,upper_quantile = timestamp_sorted_data[2].quantile([.25, .50,.75,1])
# print(lower_quantile,lower_medium_quantile,upper_medium_quantile,upper_quantile)
temp1 = timestamp_sorted_data.iloc[:6920, :]
temp2 = timestamp_sorted_data.iloc[6921:13840, :]
temp3 = timestamp_sorted_data.iloc[13841:20760, :]
temp4 = timestamp_sorted_data.iloc[20761:27683, :]
temp1_graph = create_graph(temp1.values)
temp2_graph = create_graph(temp2.values)
temp3_graph = create_graph(temp3.values)
temp4_graph = create_graph(temp4.values)

# print(temp3)

In [211]:
graph_list= [temp1_graph,temp2_graph,temp3_graph,temp4_graph]

temp_graph = None
for graph in graph_list:
    if temp_graph is None:
        temp_graph = set(graph.nodes())
    else:
        temp_graph = set(graph.nodes()) & temp_graph
# print(temp_graph)
# set(temp1_graph.nodes()

In [216]:
# every node is active node if we try to find common users in four temporal graphs 

# candidate_node_dict = defaultdict(list)
# for node in temp_graph:
#     temp_graph_closeness = nx.closeness_centrality(connected_graph, node)
#     if temp_graph_closeness <= t1:
#         label = 0 # is inactive user
#     elif temp_graph_closeness> t2:
#         label = 2 # active user
#     else:
#         label = 1 # normal user
#     candidate_node_dict[label].append(node)
# print(candidate_node_dict)


In [217]:
#----------------------------------------------------------#
#    3 groups subgraph and temporal networks over time     #
#----------------------------------------------------------#
active_users_set = [23283,33213,54964,6309,20860]
normal_users_set = [19884,18091,65584,59546,105148]
inactive_users_set = [69401,104707,162154,152286,113682]
acitve_users_graph =connected_graph.subgraph(active_users_set)
normal_users_graph =connected_graph.subgraph(normal_users_set)
inactive_users_graph =connected_graph.subgraph(inactive_users_set)

print('acitve_users_graph:''number of edges:',acitve_users_graph.number_of_edges(),'/number of nodes:',acitve_users_graph.number_of_nodes(),'/is connected or not?:',nx.is_connected(acitve_users_graph))
print('normal_users_graph:''number of edges:',normal_users_graph.number_of_edges(),'/number of nodes:',normal_users_graph.number_of_nodes(),'/is connected or not?:',nx.is_connected(normal_users_graph))
print('inactive_users_graph:''number of edges:',inactive_users_graph.number_of_edges(),'/number of nodes:',inactive_users_graph.number_of_nodes(),'/is connected or not?:',nx.is_connected(inactive_users_graph))

acitve_users_graph:number of edges: 11 /number of nodes: 5 /is connected or not?: True
normal_users_graph:number of edges: 5 /number of nodes: 5 /is connected or not?: False
inactive_users_graph:number of edges: 3 /number of nodes: 5 /is connected or not?: False


In [195]:
#----------------------------------------------------------#
#               For 1 HOP                                  #
#----------------------------------------------------------#
import json

userset_list = [active_users_set, normal_users_set, inactive_users_set]
neighbor_dict = rec_dd()
for net_index, temporal_network in zip(range(1, len(graph_list)+1), graph_list):
    for user_index, user_set in zip(range(1, len(user_list)+1),userset_list):
        for i, node in zip(range(1, len(user_set)+1), user_set):
            try:
                neighbor_list = list(nx.all_neighbors(temporal_network, node))
            except Exception as e:
                print(e)
                neighbor_list = []
            neighbor_dict[net_index][user_index][i] = neighbor_list
                
            
print(json.dumps(neighbor_dict, indent = 4))

The node 65584 is not in the graph.
The node 105148 is not in the graph.
The node 69401 is not in the graph.
The node 104707 is not in the graph.
The node 162154 is not in the graph.
The node 152286 is not in the graph.
The node 113682 is not in the graph.
The node 105148 is not in the graph.
The node 69401 is not in the graph.
The node 104707 is not in the graph.
The node 162154 is not in the graph.
The node 152286 is not in the graph.
The node 113682 is not in the graph.
The node 65584 is not in the graph.
The node 162154 is not in the graph.
The node 152286 is not in the graph.
The node 19884 is not in the graph.
The node 65584 is not in the graph.
The node 59546 is not in the graph.
The node 69401 is not in the graph.
The node 104707 is not in the graph.
The node 113682 is not in the graph.
{
    "1": {
        "1": {
            "1": [
                23283.0,
                14065.0,
                10693.0,
                1599.0,
                8945.0,
                7644.0,


In [159]:
#----------------------------------------------------------#
#               For 2 HOP                                  #
#----------------------------------------------------------#